In [1]:
import pandas as pd
import numpy as np
import sklearn as skl
from sklearn import preprocessing
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

### Indicator 1 - Renters who pay more than 50% of their income on housing

In [2]:
rent = pd.read_csv('../data/rent-as-percentage-income/ACSDT5Y2018.B25070_data_with_overlays_2020-11-19T201947.csv',
                   low_memory=False, skiprows=[1],
                   usecols=['NAME','B25070_010E','B25070_001E','B25070_011E'])
rent.rename(columns={'B25070_010E': 'paying_50p_more', 'B25070_001E': 'Total',
                    'B25070_011E': 'Total_not_comp'}, inplace = True)

rent.head()

,NAME,Total,paying_50p_more,Total_not_comp
0,"Census Tract 37, New York County, New York",789,127,61
1,"Census Tract 86.01, New York County, New York",1001,204,49
2,"Census Tract 44, New York County, New York",8145,1439,224
3,"Census Tract 86.03, New York County, New York",736,204,16
4,"Census Tract 86.02, New York County, New York",0,0,0


In [3]:
# Extract County names
rent['County'] = rent['NAME'].str.split(pat = ',',expand = True)[1]
rent.drop(['NAME'], axis=1, inplace = True)

rent

,Total,paying_50p_more,Total_not_comp,County
0,789,127,61,New York County
1,1001,204,49,New York County
2,8145,1439,224,New York County
3,736,204,16,New York County
4,0,0,0,New York County
...,...,...,...,...
2162,324,46,106,Richmond County
2163,345,84,33,Richmond County
2164,470,159,66,Richmond County
2165,275,60,59,Richmond County


In [98]:
# aggregate over counties
rent = rent.groupby(['County']).sum()

In [99]:
rent

,Total,paying_50p_more,Total_not_comp
County,,,
Bronx County,401745,132931,19630
Kings County,665526,193135,33830
New York County,575184,123027,25813
Queens County,431495,120943,24547
Richmond County,50313,14686,5613


In [50]:
rent['pay over 50'] = (rent['paying_50p_more'] / (rent['Total'] - rent['Total_not_comp'])).round(decimals=2)
rent

,Total,paying_50p_more,Total_not_comp,pay over 50
County,,,,
Bronx County,401745,132931,19630,0.35
Kings County,665526,193135,33830,0.31
New York County,575184,123027,25813,0.22
Queens County,431495,120943,24547,0.30
Richmond County,50313,14686,5613,0.33


In [51]:
# # rows with NaN - because has 0's for the columns being used for calculations
# rent[rent.isna().any(axis=1)]

# # rent table without  NaN 

# rent.dropna()

### Indicator 2 - Portion of low-income renters who have too little left ($12K) after paying housing cost

In [8]:
income = pd.read_csv('../data/low-income/ACSDT5Y2018.B25074_data_with_overlays_2020-12-10T140704.csv',
                   low_memory=False)
income.head()

,GEO_ID,NAME,B25074_001E,B25074_001M,B25074_002E,B25074_002M,B25074_003E,B25074_003M,B25074_004E,B25074_004M,B25074_005E,B25074_005M,B25074_006E,B25074_006M,B25074_007E,B25074_007M,B25074_008E,B25074_008M,B25074_009E,B25074_009M,B25074_010E,B25074_010M,B25074_011E,B25074_011M,B25074_012E,B25074_012M,B25074_013E,B25074_013M,B25074_014E,B25074_014M,B25074_015E,B25074_015M,B25074_016E,B25074_016M,B25074_017E,B25074_017M,B25074_018E,B25074_018M,B25074_019E,B25074_019M,B25074_020E,B25074_020M,B25074_021E,B25074_021M,B25074_022E,B25074_022M,B25074_023E,B25074_023M,B25074_024E,B25074_024M,B25074_025E,B25074_025M,B25074_026E,B25074_026M,B25074_027E,B25074_027M,B25074_028E,B25074_028M,B25074_029E,B25074_029M,B25074_030E,B25074_030M,B25074_031E,B25074_031M,B25074_032E,B25074_032M,B25074_033E,B25074_033M,B25074_034E,B25074_034M,B25074_035E,B25074_035M,B25074_036E,B25074_036M,B25074_037E,B25074_037M,B25074_038E,B25074_038M,B25074_039E,B25074_039M,B25074_040E,B25074_040M,B25074_041E,B25074_041M,B25074_042E,B25074_042M,B25074_043E,B25074_043M,B25074_044E,B25074_044M,B25074_045E,B25074_045M,B25074_046E,B25074_046M,B25074_047E,B25074_047M,B25074_048E,B25074_048M,B25074_049E,B25074_049M,B25074_050E,B25074_050M,B25074_051E,B25074_051M,B25074_052E,B25074_052M,B25074_053E,B25074_053M,B25074_054E,B25074_054M,B25074_055E,B25074_055M,B25074_056E,B25074_056M,B25074_057E,B25074_057M,B25074_058E,B25074_058M,B25074_059E,B25074_059M,B25074_060E,B25074_060M,B25074_061E,B25074_061M,B25074_062E,B25074_062M,B25074_063E,B25074_063M,B25074_064E,B25074_064M
0,id,Geographic Area Name,Estimate!!Total,Margin of Error!!Total,"Estimate!!Total!!Less than $10,000","Margin of Error!!Total!!Less than $10,000","Estimate!!Total!!Less than $10,000!!Less than ...","Margin of Error!!Total!!Less than $10,000!!Les...","Estimate!!Total!!Less than $10,000!!20.0 to 24...","Margin of Error!!Total!!Less than $10,000!!20....","Estimate!!Total!!Less than $10,000!!25.0 to 29...","Margin of Error!!Total!!Less than $10,000!!25....","Estimate!!Total!!Less than $10,000!!30.0 to 34...","Margin of Error!!Total!!Less than $10,000!!30....","Estimate!!Total!!Less than $10,000!!35.0 to 39...","Margin of Error!!Total!!Less than $10,000!!35....","Estimate!!Total!!Less than $10,000!!40.0 to 49...","Margin of Error!!Total!!Less than $10,000!!40....","Estimate!!Total!!Less than $10,000!!50.0 perce...","Margin of Error!!Total!!Less than $10,000!!50....","Estimate!!Total!!Less than $10,000!!Not computed","Margin of Error!!Total!!Less than $10,000!!Not...","Estimate!!Total!!$10,000 to $19,999","Margin of Error!!Total!!$10,000 to $19,999","Estimate!!Total!!$10,000 to $19,999!!Less than...","Margin of Error!!Total!!$10,000 to $19,999!!Le...","Estimate!!Total!!$10,000 to $19,999!!20.0 to 2...","Margin of Error!!Total!!$10,000 to $19,999!!20...","Estimate!!Total!!$10,000 to $19,999!!25.0 to 2...","Margin of Error!!Total!!$10,000 to $19,999!!25...","Estimate!!Total!!$10,000 to $19,999!!30.0 to 3...","Margin of Error!!Total!!$10,000 to $19,999!!30...","Estimate!!Total!!$10,000 to $19,999!!35.0 to 3...","Margin of Error!!Total!!$10,000 to $19,999!!35...","Estimate!!Total!!$10,000 to $19,999!!40.0 to 4...","Margin of Error!!Total!!$10,000 to $19,999!!40...","Estimate!!Total!!$10,000 to $19,999!!50.0 perc...","Margin of Error!!Total!!$10,000 to $19,999!!50...","Estimate!!Total!!$10,000 to $19,999!!Not computed","Margin of Error!!Total!!$10,000 to $19,999!!No...","Estimate!!Total!!$20,000 to $34,999","Margin of Error!!Total!!$20,000 to $34,999","Estimate!!Total!!$20,000 to $34,999!!Less than...","Margin of Error!!Total!!$20,000 to $34,999!!Le...","Estimate!!Total!!$20,000 to $34,999!!20.0 to 2...","Margin of Error!!Total!!$20,000 to $34,999!!20...","Estimate!!Total!!$20,000 to $34,999!!25.0 to 2...","Margin of Error!!Total!!$20,000 to $34,999!!25...","Estimate!!Total!!$20,000 to $34,999!!30.0 to 3...","Margin of Error!!Total!!$20,000 to $34,999!!30...","Estimate!!Total!!$20,000 to $34,999!!35.0 to 3...","Margin 

In [6]:
# viewing all column names to choose which ones to keep
# pd.set_option('display.max_rows', None)
# display(income.iloc[0])
# pd.reset_option('display.max_rows')

In [101]:
usecols=['NAME','S1901_C01_001E','S1901_C01_002E', 'S1901_C01_003E', 'S1901_C01_004E']
income = income[usecols]
income

,NAME,S1901_C01_001E,S1901_C01_002E,S1901_C01_003E,S1901_C01_004E
0,Geographic Area Name,Estimate!!Households!!Total,"Estimate!!Households!!Total!!Less than $10,000","Estimate!!Households!!Total!!$10,000 to $14,999","Estimate!!Households!!Total!!$15,000 to $24,999"
1,United States,119730128,6.3,4.6,9.3
2,"Bronx County, New York",499728,13.5,9.9,13.2
3,"Kings County, New York",950856,9.3,6.9,10.0
4,"New York County, New York",758133,8.2,5.6,7.6
5,"Queens County, New York",779234,5.9,4.3,8.7
6,"Richmond County, New York",166152,8.1,3.8,6.9


In [102]:
#little more cleaning
income.columns = ['NAME','total-households','p-less-10k', 'p-10-15', 'p-15-25']
income = income.drop(index = 0, axis = 0)
income

,NAME,total-households,p-less-10k,p-10-15,p-15-25
1,United States,119730128,6.3,4.6,9.3
2,"Bronx County, New York",499728,13.5,9.9,13.2
3,"Kings County, New York",950856,9.3,6.9,10.0
4,"New York County, New York",758133,8.2,5.6,7.6
5,"Queens County, New York",779234,5.9,4.3,8.7
6,"Richmond County, New York",166152,8.1,3.8,6.9


In [121]:
# ['NAME','B25070_001E','B25070_011E', 'B25070_002E', 'B25070_009E','B25070_010E']
# ['NAME','total','total-not-comp', 'less-than-10', '40-49', 'over-50']

In [122]:
# subset of the rent table needed for calculations

rent_sub = pd.read_csv('../data/rent-as-percentage-income/ACSDT5Y2018.B25070_data_with_overlays_2020-11-19T201947.csv',
                   low_memory=False, skiprows=[1],
                   usecols=['NAME','B25070_001E','B25070_011E',
                            'B25070_002E', 'B25070_009E','B25070_010E'])
rent_sub.head()

,NAME,B25070_001E,B25070_002E,B25070_009E,B25070_010E,B25070_011E
0,"Census Tract 37, New York County, New York",789,88,41,127,61
1,"Census Tract 86.01, New York County, New York",1001,102,68,204,49
2,"Census Tract 44, New York County, New York",8145,477,821,1439,224
3,"Census Tract 86.03, New York County, New York",736,135,62,204,16
4,"Census Tract 86.02, New York County, New York",0,0,0,0,0


In [123]:
# Extract County names
rent_sub['County'] = rent_sub['NAME'].str.split(pat = ',',expand = True)[1]
rent_sub.drop(['NAME'], axis=1, inplace = True)

rent_sub

,B25070_001E,B25070_002E,B25070_009E,B25070_010E,B25070_011E,County
0,789,88,41,127,61,New York County
1,1001,102,68,204,49,New York County
2,8145,477,821,1439,224,New York County
3,736,135,62,204,16,New York County
4,0,0,0,0,0,New York County
...,...,...,...,...,...,...
2162,324,11,9,46,106,Richmond County
2163,345,28,0,84,33,Richmond County
2164,470,17,19,159,66,Richmond County
2165,275,0,21,60,59,Richmond County


In [125]:
# aggregate over counties
rent_sub = rent_sub.groupby(['County']).sum()
rent_sub

,B25070_001E,B25070_002E,B25070_009E,B25070_010E,B25070_011E
County,,,,,
Bronx County,401745,12152,36347,132931,19630
Kings County,665526,28114,54147,193135,33830
New York County,575184,51242,43667,123027,25813
Queens County,431495,13971,38790,120943,24547
Richmond County,50313,1818,3425,14686,5613


In [126]:
rent_sub.columns = ['total','total-not-comp', 'less-than-10', '40-49', 'over-50']
rent_sub

,total,total-not-comp,less-than-10,40-49,over-50
County,,,,,
Bronx County,401745,12152,36347,132931,19630
Kings County,665526,28114,54147,193135,33830
New York County,575184,51242,43667,123027,25813
Queens County,431495,13971,38790,120943,24547
Richmond County,50313,1818,3425,14686,5613


In [127]:
income

,NAME,total-households,p-less-10k,p-10-15,p-15-25
1,United States,119730128,6.3,4.6,9.3
2,"Bronx County, New York",499728,13.5,9.9,13.2
3,"Kings County, New York",950856,9.3,6.9,10.0
4,"New York County, New York",758133,8.2,5.6,7.6
5,"Queens County, New York",779234,5.9,4.3,8.7
6,"Richmond County, New York",166152,8.1,3.8,6.9


In [ ]:
#FINAL TABLE


### Indicator 3 - UI coverage rate

#### Method
\begin{align*}
&\text{UI coverage rate} = \frac{\text{private-sector workers in the UI program}}{\text{number of workers in the private for-profit and nonprofit sectors}}\\
\\
&\text{non-UI coverage rate} = 1-\text{UI coverage rate}
\end{align*}

where the private-sector workers in the UI program comes from **Total number of jobs (C000)** in Private Jobs in  Longitudinal Employer–Household Dynamics (LEHD) for 2013‒17 (the five most recent years available), and the estimated labor force comes from **Private wage and salary workers (DP03_0047E)** in the corresponding 2013‒17 ACS. 

Data collection and cleanning details can be found in **UCLA_UI.ipnb**

In [13]:
acs = pd.read_csv('../data/UI_coverage/ACS_DP03_cleaned.csv', low_memory=False)
rac_avg = pd.read_csv('../data/UI_coverage/ny_rac_cleaned.csv', low_memory=False)

In [17]:
ui_coverage = acs.merge(rac_avg, left_on='GEO_ID', right_on='trct', how='left')
ui_coverage['ui_rate'] = ui_coverage['total_ui_jobs'] / ui_coverage['total_jobs']
ui_coverage = ui_coverage[['GEO_ID', 'NAME', 'total_ui_jobs', 'total_jobs', 'ui_rate']]

print('# ui rate>1:', sum(ui_coverage['ui_rate']>1))
print(ui_coverage[~np.isinf(ui_coverage.ui_rate)].ui_rate.describe())

# ui rate>1: 1016
count    2117.000000
mean        1.076616
std         0.913469
min         0.444891
25%         0.872669
50%         0.982034
75%         1.108605
max        31.350000
Name: ui_rate, dtype: float64


In [18]:
ui_coverage.loc[ui_coverage['ui_rate']>1, 'ui_rate'] = 1
ui_coverage.head()

,GEO_ID,NAME,total_ui_jobs,total_jobs,ui_rate
0,36061000100,"Census Tract 1, New York County, New York",175.20,0.0,1.0
1,36061000201,"Census Tract 2.01, New York County, New York",1087.80,892.0,1.0
2,36061000202,"Census Tract 2.02, New York County, New York",2427.25,2365.0,1.0
3,36061000500,"Census Tract 5, New York County, New York",167.35,0.0,1.0
4,36061000600,"Census Tract 6, New York County, New York",3531.35,2481.0,1.0


### Indicator 3 - Unemployment rate

In [23]:
employment = pd.read_csv('../data/employment_ACSDP03_2018.csv',
                         low_memory=False, skiprows=[1],
                         usecols=['GEO_ID','NAME','DP03_0003E','DP03_0005E','DP03_0037E'])

In [24]:
employment.head()

,GEO_ID,NAME,DP03_0003E,DP03_0005E,DP03_0037E
0,1400000US36005000100,"Census Tract 1, Bronx County, New York",0,0,0
1,1400000US36005000200,"Census Tract 2, Bronx County, New York",1873,295,57
2,1400000US36005000400,"Census Tract 4, Bronx County, New York",3052,244,207
3,1400000US36005001600,"Census Tract 16, Bronx County, New York",2477,164,306
4,1400000US36005001900,"Census Tract 19, Bronx County, New York",1451,192,103


In [25]:
employment['DP03_0003E']=employment['DP03_0003E'].astype(float)
employment['DP03_0005E']=employment['DP03_0005E'].astype(float)

In [26]:
employment['unemployment rate'] = (100 * employment['DP03_0005E'] / employment['DP03_0003E']).round(decimals=2)
employment.head()

,GEO_ID,NAME,DP03_0003E,DP03_0005E,DP03_0037E,unemployment rate
0,1400000US36005000100,"Census Tract 1, Bronx County, New York",0.0,0.0,0,NaN
1,1400000US36005000200,"Census Tract 2, Bronx County, New York",1873.0,295.0,57,15.75
2,1400000US36005000400,"Census Tract 4, Bronx County, New York",3052.0,244.0,207,7.99
3,1400000US36005001600,"Census Tract 16, Bronx County, New York",2477.0,164.0,306,6.62
4,1400000US36005001900,"Census Tract 19, Bronx County, New York",1451.0,192.0,103,13.23


### Indicator 4 - employment in impacted sectors (retail sales, hospitality service, personal care)